# 导入相关库

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import datetime
from tqdm import trange
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import warnings
import time
from matplotlib import pyplot as plt 
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb
from sklearn.model_selection import KFold, RepeatedKFold
from xgboost import *
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.preprocessing import *
from sklearn.linear_model import  *
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sympy import *
from xgboost import plot_importance
warnings.filterwarnings('ignore')

# 工具函数

In [2]:
dict={}
dict['四川省']=32617
dict['安徽省']=32001
dict['福建省']=58145
dict['河北省']=38909
dict['贵州省']=23151
dict['山东省']=56885
dict['黑龙江省']=37697
dict['广东省']=58833
dict['北京']=94648
dict['重庆']=43223
dict['浙江省']=68805
dict['辽宁省']=61996
dict['河南省']=34211
dict['海南省']=35663
dict['湖南省']=36943
dict['甘肃省']=24539
dict['陕西省']=43117
dict['内蒙古自治区']=67836
dict['江苏省']=75354
dict['天津']=100105
dict['上海']=90993
dict['云南省']=25322
dict['湖北省']=42826
dict['江西省']=31930
dict['吉林省']=47428
dict['山西省']=34984
dict['宁夏回族自治区']=39613
dict['西藏自治区']=26326
dict['新疆维吾尔自治区']=37553
dict['广西壮族自治区']=30741
dict['青海省']=36875
ave=0
total=0
for key in dict:
    total+=dict[key]
ave = total/len(dict)

In [3]:
'''数据归一化'''
def std_data(submit,train):
    mean_list=submit.mean()
    std_list=submit.std()
    submit_std = submit
    train_std = train
    for col in submit.columns:
        if col=='customer_id':
            print('get it!')
            pass
        else:
            submit_std[col] = (submit[col] - mean_list[col]) / std_list[col]
            train_std[col] = (train[col] - mean_list[col]) / std_list[col]
    return submit_std,train_std 

In [4]:
#查看结果的最大值，最小值，平均值
def show(result):
    print('max: ',str(max(result)))
    print('min: ',str(min(result)))
    print('mean: ',str((sum(result))/(len(result))))
    print(0)

In [5]:
#官方的loss评估
def logloss(y_true, y_pred,deta = 3.6, eps=1e-15):
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss / len(y_true)

In [6]:
"""
卡方特征选取
"""
def chi_square_test(X,y,select_k=10):
    if select_k >= 1:
        sel_ = SelectKBest(k=select_k).fit(X,y)
        col = X.columns[sel_.get_support()]
    elif 0 < select_k < 1:
        sel_ = SelectPercentile(percentile=select_k*100).fit(X,y)
        col = X.columns[sel_.get_support()]   
    else:
        raise ValueError("select_k must be a positive number")  
    
    return col

In [8]:
'''
把结果保存成指定格式的文件
示例：save_result2csv(y_submit,submit,'./round1_diac2019_test.csv')
其中y_submit为模型输出结果，submit为读入数据时生成的一个变量，第三个参数为保存的文件路劲
'''
def save_result2csv(ys_submit,submit,csv_name):
    all_customers = pd.DataFrame(trian[['customer_id']]).drop_duplicates(['customer_id']).dropna()
    submits_df = submit[['customer_id']]
    submits_df['result'] = ys_submit
    all_customers = pd.merge(all_customers,submits_df,on=['customer_id'],how='left',copy=False)
    all_customers = all_customers.sort_values(['customer_id'])
    all_customers['customer_id'] = all_customers['customer_id'].astype('int64')
    all_customers['result'] = all_customers['result'].fillna(0)
    all_customers.to_csv(csv_name,index=False)

In [9]:
# 读取原始数据
trian = pd.read_csv('../Data/round1_diac2019_train.csv',low_memory=False)
all_customer = pd.DataFrame(trian[['customer_id']]).drop_duplicates(['customer_id']).dropna()
print(all_customer.shape)
print('train date gap',trian.order_pay_time.max(),trian.order_pay_time.min())

'''
提供了2013年全年的数据，2013-12-31 23:59:44 2013-01-01 00:00:18
'''
trian['order_pay_time'] = pd.to_datetime(trian['order_pay_time'])
trian['order_pay_date'] = trian['order_pay_time'].dt.date
validata_date_begin = trian['order_pay_date'].max() - datetime.timedelta(days=180)

trian["GDP"]=ave
for key in dict:
    trian["GDP"][trian["customer_province"]==key]=dict[key]

train_history = trian[(trian['order_pay_date'].astype(str)<='2013-07-03')]
online_history = trian[(trian['order_pay_date'].astype(str)<='2013-12-31')]
train_label = trian[trian['order_pay_date'].astype(str)>='2013-07-04']

(929807, 1)
train date gap 2013-12-31 23:59:44 2013-01-01 00:00:18


# 相关数据处理

In [10]:
# 删除类别唯一的特征
for df in [train_history,online_history]:
    df.drop(['order_detail_id','order_id','goods_id','member_id'], axis=1, inplace=True)

In [11]:
# 删除某一类别占比超过90%的列
for df in [train_history,online_history]:
    good_cols = list(df.columns)
    for col in df.columns:
        rate = df[col].value_counts(normalize=True, dropna=False).values[0]
        if rate > 0.9:
            good_cols.remove(col)
            print(col,rate)

order_total_discount 0.9973312428068654
order_status 0.9455239937951395
order_detail_status 0.9441710821625088
order_detail_goods_num 0.9926961304874004
order_total_discount 0.9873889141516714
order_status 0.9128238881064168
order_detail_status 0.9116224583836309
order_detail_goods_num 0.9924087916386773


In [12]:
#删除异常值
#trian = trian[trian[]]
train_history = train_history[train_history['order_amount']<=2000]
train_history = train_history[train_history['order_total_payment']<=2000]
#online_history = online_history[online_history['order_amount']<=2000]
#online_history = online_history[online_history['order_total_payment']<=2000]

In [13]:
#添加一些特征 
train_history['goods_num'] = train_history['order_total_num']*train_history['order_count']
train_label['goods_num'] = train_label['order_total_num']*train_label['order_count']
online_history['goods_num'] = online_history['order_total_num']*online_history['order_count']

train_history['dis_rate'] = (train_history['order_total_discount'])/train_history['order_amount']
train_label['dis_rate'] = (train_label['order_total_discount'])/train_label['order_amount']
online_history['dis_rate'] = (online_history['order_total_discount'])/online_history['order_amount']

In [14]:
for df in [train_history,train_label,online_history]:
    df['log_order_amount'] = np.log(df['order_amount'].values+1)
    df['log_goods_price'] = np.log(df['goods_price'].values+1)
    df['pay_deleta_time'] = ( pd.to_datetime(df['goods_list_time'])  - pd.to_datetime(df['order_pay_time']))
    df['goods_list_time'] = ( pd.to_datetime(df['goods_delist_time'])-pd.to_datetime(df['goods_list_time']))
    df['pay_deleta_time'] = df['pay_deleta_time'].dt.days+1
    df['goods_list_time'] = df['goods_list_time'].dt.days+1

# 特征工程

In [17]:
# 简单的特征生成部分代码
def make_feature_and_label(date1,date2,isSubmit):
    date1['count'] = 1
    # 统计这个用户出现了多少次
    customer_id = date1.groupby(['customer_id'],as_index=False)['count'].agg({'count':'count'})
    # 统计这个用户购买商品的价格信息
    good_price = date1.groupby(['customer_id'],as_index=False)['goods_price'].agg({'goods_price_max':'max',
                                                                                    'goods_price_min':'min',
                                                                                    'goods_price_mean':'mean'})
    
    #添加用户性别
    customer_gender= date1.groupby(['customer_id'],as_index=False)['customer_gender'].mean()
    customer_gender = customer_gender.fillna(0)
    
    
    #添加用户所属城市
    class_le = LabelEncoder()
    date1['customer_city'].fillna('未知',inplace=True)
    date1['customer_city_l'] = class_le.fit_transform(date1['customer_city'])
    customer_city_l = date1.groupby(['customer_id'],as_index=False)['customer_city_l'].max()
    
    #添加用户所属省份
    date1['customer_province'].fillna('未知',inplace=True)
    date1['customer_province_l'] = class_le.fit_transform(date1['customer_province'])
    customer_province_l = date1.groupby(['customer_id'],as_index=False)['customer_province_l'].max()
    
    #统计该用户是不是会员
    is_member = date1.groupby(['customer_id'],as_index=False)['is_member_actived'].median()
    is_member = is_member.fillna(0)
    
    #统计用户购物车内商品数量
    goods = date1.groupby(['customer_id'],as_index=False)['order_count'].max()
    goods=goods.fillna(0)
    
    #统计用户花了多少钱及多少个
    order_total_num = date1.groupby(['customer_id'],as_index=False)['order_total_num'].mean()
    order_total_num = order_total_num.fillna(0)
    
    order_count = date1.groupby(['customer_id'],as_index=False)['order_count'].mean()
    order_count = order_count.fillna(0)
    
    total_num = order_count*order_total_num
    
    order_amount = date1.groupby(['customer_id'],as_index=False)['order_amount'].max()
    order_amount = order_amount.fillna(0)
    
    order_total_payment = date1.groupby(['customer_id'],as_index=False)['order_total_payment'].mean()
    order_total_payment = order_total_payment.fillna(0)
    order_total_payment_level = order_total_payment
    order_total_payment_level['order_total_payment'] = order_total_payment['order_total_payment']>293
      
    
    goods_num = date1.groupby(['customer_id'],as_index=False)['goods_num'].mean()
    goods_num.fillna(0)
     
    gdp = date1.groupby(['customer_id'],as_index=False)['GDP'].mean()
    
    goods_list_time = date1.groupby(['customer_id'],as_index=False)['goods_list_time'].agg({'goods_list_time_last':'max',
                                                                                           'goods_list_time_mean':'mean'})
    
        
    pay_deleta_time = date1.groupby(['customer_id'],as_index=False)['pay_deleta_time'].agg({'pay_deleta_time_last':'max',
                                                                                            'pay_deleta_time_first':'min',
                                                                                           'pay_deleta_time_mean':'mean'})
    
    # 统计这个用户的订单最后一次购买时间
    last_time = date1.groupby(['customer_id'],as_index=False)['order_pay_date'].agg({'order_pay_date_last':'max','order_pay_date_first':'min'})
    # 当然这里面还可以构造更多的特征 
    '''
                    在这里疯狂加特征！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！！
    '''
    data = pd.merge(customer_id,good_price,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,is_member,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,customer_gender,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,gdp,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,pay_deleta_time,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,goods_list_time,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,goods_num,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,order_total_num,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,order_total_payment_level,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,customer_province_l,on=['customer_id'],how='left',copy=False)
    data = pd.merge(data,last_time,on=['customer_id'],how='left',copy=False)
    data['long_time'] = pd.to_datetime(data['order_pay_date_last']) - pd.to_datetime(data['order_pay_date_first'])
    data['long_time'] = data['long_time'].dt.days + 1
    del data['order_pay_date_first']
    if isSubmit==False:
        data['order_pay_date_last'] = pd.to_datetime(date2['order_pay_date'].min()) - pd.to_datetime(data['order_pay_date_last'])
        data['order_pay_date_last'] = data['order_pay_date_last'].dt.days + 1
        data['label'] = 0
        data.loc[data['customer_id'].isin(list(date2['customer_id'].unique())),'label'] = 1
        print(data['label'].mean())
    else:
        data['order_pay_date_last'] = pd.to_datetime('2013-12-31') - pd.to_datetime(data['order_pay_date_last'])
        data['order_pay_date_last'] = data['order_pay_date_last'].dt.days + 1
    print(data.shape)
    return data

In [18]:
train = make_feature_and_label(train_history,train_label,False)
submit = make_feature_and_label(online_history,None,True)

0.1627399848929293
(374659, 20)
(929807, 19)


In [16]:
y = train.pop('label')
feature = [x for x in train.columns if x not in ['customer_id']]
X = train[feature]

# 第一次训练

In [11]:
#第一次训练，利用卡方特征选取18个特征进行训练LGB和XGB
x_columns = chi_square_test(X, y, 18)
print(x_columns)
X_train, X_valid, y_train, y_valid = train_test_split(X[x_columns], y, test_size=0.25, random_state=42,stratify=y)
submit_df = submit[['customer_id']]
X_submit0 = submit[feature]
X_submit = X_submit0[x_columns]

Index(['count', 'goods_price_mean', 'goods_price_min', 'goods_price_max',
       'is_member_actived', 'customer_gender', 'GDP', 'pay_deleta_time_last',
       'pay_deleta_time_mean', 'pay_deleta_time_first', 'goods_list_time_last',
       'goods_list_time_mean', 'goods_num', 'order_total_num',
       'order_total_payment', 'customer_province_l', 'order_pay_date_last',
       'long_time'],
      dtype='object')


In [25]:
#针对本次问题对loss function做了改动
def customObj(real, predict):
    deta = 3.6
    grad = -1*( ((deta*real)/predict) ) - ( (real - 1)/( 1-predict ) )
    hess = np.power(np.abs(grad), 1)
    return grad, hess

In [24]:
#8折 训练rf 
n_splits = 8
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_rf = np.zeros(len(X_submit))
y_pp_rf_stacking = np.zeros(len(y_train))
for train_index, test_index in kf.split(X_train):
    print ( ">>>", train_index )
    rf = RandomForestClassifier(n_estimators=50, max_depth=8,random_state=10,class_weight='balanced_subsample')
        
    rf_model = rf.fit(X_train.iloc[train_index], y_train.iloc[train_index]) 

    #plot_importance(xgboost_model)
    y_pred = rf_model.predict(X_train.iloc[test_index]) 
    y_predprob = rf_model.predict_proba(X_train.iloc[test_index])[:, 1] 
    
    y_pp_rf_stacking[test_index] = y_predprob 
    
    print(metrics.confusion_matrix(y_train.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_rf += rf_model.predict_proba(X_submit)[:, 1] / n_splits 

>>> [     1      2      3 ... 280990 280991 280992]
[[22053  7473]
 [ 2216  3383]]
Accuracy : 0.7242
AUC Score (Train): 0.732725
logloss SCore: 0.851799
>>> [     0      1      2 ... 280991 280992 280993]
[[22161  7148]
 [ 2326  3490]]
Accuracy : 0.7303
AUC Score (Train): 0.733098
logloss SCore: 0.859362
>>> [     0      1      2 ... 280991 280992 280993]
[[22146  7258]
 [ 2294  3426]]
Accuracy : 0.728
AUC Score (Train): 0.734311
logloss SCore: 0.853793
>>> [     0      1      2 ... 280990 280991 280993]
[[22070  7359]
 [ 2337  3358]]
Accuracy : 0.7239
AUC Score (Train): 0.726135
logloss SCore: 0.860625
>>> [     0      1      4 ... 280990 280992 280993]
[[22048  7378]
 [ 2312  3386]]
Accuracy : 0.7241
AUC Score (Train): 0.727367
logloss SCore: 0.860627
>>> [     0      1      2 ... 280991 280992 280993]
[[22124  7315]
 [ 2291  3394]]
Accuracy : 0.7265
AUC Score (Train): 0.731978
logloss SCore: 0.854650
>>> [     0      2      3 ... 280991 280992 280993]
[[22054  7315]
 [ 2382  3373]]


In [28]:
#8折 训练xgb 这里没有在数据中增强正样本比例，而是跟换了loss function
n_splits = 8
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_xgb = np.zeros(len(X_submit))
y_pp_xgb_stacking = np.zeros(len(y_train))
for train_index, test_index in kf.split(X_train):
    print ( ">>>", train_index )
    xgboost = xgb.XGBClassifier(learning_rate =0.01,
                                 n_estimators=250,
                                 max_depth=8,
                                 min_child_weight=1,
                                 eta = 0.01,
                                 gamma=0,
                                 subsample=0.8,
                                 colsample_bytree=0.8,
                                 objective= customObj,
                                 nthread=6,
                                 scale_pos_weight=1.2,
                                 seed=66)
    xgboost_model = xgboost.fit(X_train.iloc[train_index], y_train.iloc[train_index]) 

    #plot_importance(xgboost_model)
    y_pred = xgboost_model.predict(X_train.iloc[test_index]) 
    y_predprob = xgboost_model.predict_proba(X_train.iloc[test_index])[:, 1] 
    
    y_pp_xgb_stacking[test_index] = y_predprob 
    
    print(metrics.confusion_matrix(y_train.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_xgb += xgboost_model.predict_proba(X_submit)[:, 1] / n_splits 

>>> [     1      2      3 ... 280990 280991 280992]
[[24773  4753]
 [ 2854  2745]]
Accuracy : 0.7834
AUC Score (Train): 0.733810
logloss SCore: 0.844603
>>> [     0      1      2 ... 280991 280992 280993]
[[24969  4340]
 [ 3031  2785]]
Accuracy : 0.7901
AUC Score (Train): 0.732960
logloss SCore: 0.857281
>>> [     0      1      2 ... 280991 280992 280993]
[[24978  4426]
 [ 2928  2792]]
Accuracy : 0.7906
AUC Score (Train): 0.735260
logloss SCore: 0.849133
>>> [     0      1      2 ... 280990 280991 280993]
[[24731  4698]
 [ 2928  2767]]
Accuracy : 0.7829
AUC Score (Train): 0.727755
logloss SCore: 0.853516
>>> [     0      1      4 ... 280990 280992 280993]
[[24735  4691]
 [ 2946  2752]]
Accuracy : 0.7826
AUC Score (Train): 0.727615
logloss SCore: 0.854285
>>> [     0      1      2 ... 280991 280992 280993]
[[24876  4563]
 [ 2929  2756]]
Accuracy : 0.7867
AUC Score (Train): 0.732903
logloss SCore: 0.849702
>>> [     0      2      3 ... 280991 280992 280993]
[[24783  4586]
 [ 2981  2774]]

In [38]:
#8折 训练lgb 这里在数据层面增强了正样本比例，但没有更换loss function
n_splits = 8
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_lgb = np.zeros(len(X_submit))
y_pp_lgb_stacking = np.zeros(len(y_train))
for train_index, test_index in kf.split(X_train):
    print ( ">>>", train_index )
    lightgbm = lgb.LGBMClassifier(learning_rate =0.01,
                                 n_estimators=250,
                                 max_depth=12,
                                 min_child_weight=1,
                                 eta = 0.01,
                                 alpha = 0.01,
                                 gamma=0.0,
                                 subsample=0.7,
                                 colsample_bytree=0.7,
                                 nthread=6,
                                 scale_pos_weight=3.5,
                                 seed=66)
    lightgbm_model = lightgbm.fit(X_train.iloc[train_index], y_train.iloc[train_index]) 

    y_pred = lightgbm_model.predict(X_train.iloc[test_index]) 
    y_predprob = lightgbm_model.predict_proba(X_train.iloc[test_index])[:, 1] 
    
    y_pp_lgb_stacking[test_index] = y_predprob
    
    print(metrics.confusion_matrix(y_train.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_lgb += lightgbm_model.predict_proba(X_submit)[:, 1] / n_splits 

>>> [     1      2      3 ... 280990 280991 280992]
[[25843  3683]
 [ 3162  2437]]
Accuracy : 0.8051
AUC Score (Train): 0.733684
logloss SCore: 0.833440
>>> [     0      1      2 ... 280991 280992 280993]
[[25900  3409]
 [ 3346  2470]]
Accuracy : 0.8077
AUC Score (Train): 0.734026
logloss SCore: 0.849109
>>> [     0      1      2 ... 280991 280992 280993]
[[25836  3568]
 [ 3187  2533]]
Accuracy : 0.8077
AUC Score (Train): 0.735134
logloss SCore: 0.839221
>>> [     0      1      2 ... 280990 280991 280993]
[[25768  3661]
 [ 3246  2449]]
Accuracy : 0.8034
AUC Score (Train): 0.728489
logloss SCore: 0.844683
>>> [     0      1      4 ... 280990 280992 280993]
[[25781  3645]
 [ 3268  2430]]
Accuracy : 0.8032
AUC Score (Train): 0.728263
logloss SCore: 0.845450
>>> [     0      1      2 ... 280991 280992 280993]
[[25914  3525]
 [ 3250  2435]]
Accuracy : 0.8071
AUC Score (Train): 0.733058
logloss SCore: 0.840016
>>> [     0      2      3 ... 280991 280992 280993]
[[25851  3518]
 [ 3266  2489]]

# 第二次训练

In [39]:
#第二次训练，利用卡方特征选取8个特征进行训练LGB,XGB,RF
x_columns0 = chi_square_test(X, y, 9)
print(x_columns0)
X_train0, X_valid0, y_train0, y_valid0 = train_test_split(X[x_columns0], y, test_size=0.25, random_state=42,stratify=y)
submit_df = submit[['customer_id']]
X_submit00 = submit[feature]
X_submit01 = X_submit00[x_columns0]

Index(['count', 'is_member_actived', 'customer_gender',
       'pay_deleta_time_first', 'goods_list_time_last', 'goods_num',
       'order_total_payment', 'order_pay_date_last', 'long_time'],
      dtype='object')


In [42]:
#8折 训练RF
n_splits = 8
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_rf0 = np.zeros(len(X_submit))
y_pp_rf_stacking0 = np.zeros(len(y_train0))
for train_index, test_index in kf.split(X_train0):
    print ( ">>>", train_index )
    rf0 = RandomForestClassifier(n_estimators=20, max_depth=12,random_state=10,class_weight='balanced')
       
    rf_model0 = rf0.fit(X_train0.iloc[train_index], y_train0.iloc[train_index]) 

    #plot_importance(xgboost_model)
    y_pred = rf_model0.predict(X_train0.iloc[test_index]) 
    y_predprob = rf_model0.predict_proba(X_train0.iloc[test_index])[:, 1] 
    
    y_pp_rf_stacking0[test_index] = y_predprob 
    
    print(metrics.confusion_matrix(y_train0.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train0.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train0.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train0.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_rf0 += rf_model0.predict_proba(X_submit01)[:, 1] / n_splits 

>>> [     1      2      3 ... 280990 280991 280992]
[[23013  6513]
 [ 2463  3136]]
Accuracy : 0.7445
AUC Score (Train): 0.725132
logloss SCore: 0.852528
>>> [     0      1      2 ... 280991 280992 280993]
[[23082  6227]
 [ 2599  3217]]
Accuracy : 0.7487
AUC Score (Train): 0.722514
logloss SCore: 0.863438
>>> [     0      1      2 ... 280991 280992 280993]
[[22937  6467]
 [ 2490  3230]]
Accuracy : 0.745
AUC Score (Train): 0.726793
logloss SCore: 0.853944
>>> [     0      1      2 ... 280990 280991 280993]
[[22963  6466]
 [ 2558  3137]]
Accuracy : 0.7431
AUC Score (Train): 0.719935
logloss SCore: 0.860839
>>> [     0      1      4 ... 280990 280992 280993]
[[22943  6483]
 [ 2551  3147]]
Accuracy : 0.7428
AUC Score (Train): 0.719467
logloss SCore: 0.862822
>>> [     0      1      2 ... 280991 280992 280993]
[[22930  6509]
 [ 2534  3151]]
Accuracy : 0.7425
AUC Score (Train): 0.722479
logloss SCore: 0.857567
>>> [     0      2      3 ... 280991 280992 280993]
[[22782  6587]
 [ 2568  3187]]


In [46]:
#8折 训练xgb 这里没有在数据中增强正样本比例，而是跟换了loss function
n_splits = 8
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_xgb0 = np.zeros(len(X_submit01))
y_pp_xgb_stacking0 = np.zeros(len(y_train0))
for train_index, test_index in kf.split(X_train0):
    print ( ">>>", train_index )
    xgboost0 = xgb.XGBClassifier(learning_rate =0.01,
                                 n_estimators=300,
                                 max_depth=8,
                                 min_child_weight=1,
                                 eta = 0.01,
                                 gamma=0.01,
                                 subsample=0.8,
                                 colsample_bytree=0.8,
                                 objective= customObj,
                                 nthread=6,
                                 scale_pos_weight=1,
                                 seed=66)
    xgboost_model0 = xgboost0.fit(X_train0.iloc[train_index], y_train0.iloc[train_index]) 

    #plot_importance(xgboost_model)
    y_pred = xgboost_model0.predict(X_train0.iloc[test_index]) 
    y_predprob = xgboost_model0.predict_proba(X_train0.iloc[test_index])[:, 1] 
    
    y_pp_xgb_stacking0[test_index]=y_predprob
    
    print(metrics.confusion_matrix(y_train0.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train0.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train0.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train0.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_xgb0 += xgboost_model0.predict_proba(X_submit01)[:, 1] / n_splits 

>>> [     1      2      3 ... 280990 280991 280992]
[[25104  4422]
 [ 2950  2649]]
Accuracy : 0.7901
AUC Score (Train): 0.730351
logloss SCore: 0.842376
>>> [     0      1      2 ... 280991 280992 280993]
[[25209  4100]
 [ 3133  2683]]
Accuracy : 0.7941
AUC Score (Train): 0.729703
logloss SCore: 0.855509
>>> [     0      1      2 ... 280991 280992 280993]
[[25139  4265]
 [ 2993  2727]]
Accuracy : 0.7934
AUC Score (Train): 0.731339
logloss SCore: 0.846556
>>> [     0      1      2 ... 280990 280991 280993]
[[25109  4320]
 [ 3059  2636]]
Accuracy : 0.7899
AUC Score (Train): 0.724204
logloss SCore: 0.851460
>>> [     0      1      4 ... 280990 280992 280993]
[[25156  4270]
 [ 3064  2634]]
Accuracy : 0.7912
AUC Score (Train): 0.724866
logloss SCore: 0.852156
>>> [     0      1      2 ... 280991 280992 280993]
[[25210  4229]
 [ 3045  2640]]
Accuracy : 0.7929
AUC Score (Train): 0.728443
logloss SCore: 0.847872
>>> [     0      2      3 ... 280991 280992 280993]
[[25154  4215]
 [ 3082  2673]]

In [88]:
#8折 训练lgb 这里在数据层面增强了正样本比例，但没有更换loss function
n_splits = 8
kf = KFold(n_splits=n_splits, shuffle=True, random_state=1234)
y_pp_lgb0 = np.zeros(len(X_submit01))
y_pp_lgb_stacking0 = np.zeros(len(y_train0))
for train_index, test_index in kf.split(X_train0):
    print ( ">>>", train_index )
    lightgbm0 = lgb.LGBMClassifier(learning_rate =0.03,
                                 n_estimators=250,
                                 max_depth=9,
                                 min_child_weight=1,
                                 eta = 0.03,
                                 alpha = 0.1,
                                 gamma=0.1,
                                 subsample=0.8,
                                 colsample_bytree=0.8,
                                 nthread=6,
                                 scale_pos_weight=2,
                                 seed=66)
    lightgbm_model0 = lightgbm0.fit(X_train0.iloc[train_index], y_train0.iloc[train_index]) 

    y_pred = lightgbm_model0.predict(X_train0.iloc[test_index]) 
    y_predprob = lightgbm_model0.predict_proba(X_train0.iloc[test_index])[:, 1] 
    
    y_pp_lgb_stacking0[test_index] = y_predprob
    
    print(metrics.confusion_matrix(y_train0.iloc[test_index].values, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(y_train0.iloc[test_index].values, y_pred))  
    auc = metrics.roc_auc_score(y_train0.iloc[test_index], y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(y_train0.iloc[test_index],y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    y_pp_lgb0 += lightgbm_model0.predict_proba(X_submit01)[:, 1] / n_splits 

>>> [     1      2      3 ... 280990 280991 280992]
[[27473  2053]
 [ 3794  1805]]
Accuracy : 0.8335
AUC Score (Train): 0.730949
logloss SCore: 0.874133
>>> [     0      1      2 ... 280991 280992 280993]
[[27351  1958]
 [ 3979  1837]]
Accuracy : 0.831
AUC Score (Train): 0.730466
logloss SCore: 0.899833
>>> [     0      1      2 ... 280991 280992 280993]
[[27424  1980]
 [ 3861  1859]]
Accuracy : 0.8337
AUC Score (Train): 0.731560
logloss SCore: 0.883443
>>> [     0      1      2 ... 280990 280991 280993]
[[27277  2152]
 [ 3835  1860]]
Accuracy : 0.8295
AUC Score (Train): 0.725066
logloss SCore: 0.890197
>>> [     0      1      4 ... 280990 280992 280993]
[[27295  2131]
 [ 3877  1821]]
Accuracy : 0.8289
AUC Score (Train): 0.724952
logloss SCore: 0.891657
>>> [     0      1      2 ... 280991 280992 280993]
[[27432  2007]
 [ 3867  1818]]
Accuracy : 0.8328
AUC Score (Train): 0.729062
logloss SCore: 0.885214
>>> [     0      2      3 ... 280991 280992 280993]
[[27412  1957]
 [ 3955  1800]]


# Stacking 6次训练的模型

In [93]:
# 将两次训练的lgb,xgb,rf的结果进行stacking
train_stack = np.vstack([y_pp_lgb_stacking,y_pp_lgb_stacking0,y_pp_xgb_stacking,y_pp_xgb_stacking0,y_pp_rf_stacking,y_pp_rf_stacking0]).transpose()
test_stack = np.vstack([y_pp_lgb,y_pp_lgb0,y_pp_xgb,y_pp_xgb0,y_pp_rf,y_pp_rf0]).transpose()

#train_stack = np.vstack([y_pp_lgb_stacking,y_pp_lgb_stacking0,y_pp_xgb_stacking,y_pp_xgb_stacking0]).transpose()
#test_stack = np.vstack([y_pp_lgb,y_pp_lgb0,y_pp_xgb,y_pp_xgb0]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=666)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])
predictions_final = np.zeros(X_submit.shape[0])
y_predall = np.zeros(train_stack.shape[0])
y_predproball = np.zeros(train_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,y_train)):
    print("fold {}".format(fold_))
    t1 = time.time()
    trn_data, trn_y = train_stack[trn_idx], y_train.iloc[trn_idx]
    val_data, val_y = train_stack[val_idx], y_train.iloc[val_idx]


    clf_5 =  lgb.LGBMClassifier(learning_rate =0.01,
                                 n_estimators=200,
                                 max_depth=15,
                                 min_child_weight=1,
                                 eta = 0.01,
                                 alpha = 0.01,
                                 gamma=0.0,
                                 subsample=0.8,
                                 colsample_bytree=0.8,
                                 nthread=6,
                                 scale_pos_weight=3,
                                 seed=66)
                             
    #clf_5 = LinearDiscriminantAnalysis()
    #clf_5 = LogisticRegression()
    
    clf_5.fit(trn_data, trn_y)
    
    
    y_pred = clf_5.predict(val_data) 
    y_predprob = clf_5.predict_proba(val_data)[:, 1] 
    y_predall[val_idx] = y_pred
    y_predproball[val_idx] = y_predprob
      
    print(metrics.confusion_matrix(val_y, y_pred))
    print("Accuracy : %.4g" % metrics.accuracy_score(val_y, y_pred))  
    auc = metrics.roc_auc_score(val_y, y_predprob)
    print("AUC Score (Train): %f" % auc) 
    loglossscore = logloss(val_y,y_predprob)
    print("logloss SCore: %f" %loglossscore)
    
    
    oof_stack[val_idx] = np.array(clf_5.predict_proba(val_data)[:,1])
    predictions += np.array(clf_5.predict_proba(test_stack)[:,1])/ 10
    t2 = time.time()
    print('escape time:',str(t2-t1))
print(logloss(y_train, oof_stack))

fold 0
[[42543  4285]
 [ 5952  3419]]
Accuracy : 0.8178
AUC Score (Train): 0.731909
logloss SCore: 0.869212
escape time: 3.563441514968872
fold 1
[[42576  4612]
 [ 5538  3473]]
Accuracy : 0.8194
AUC Score (Train): 0.729905
logloss SCore: 0.849950
escape time: 3.6103453636169434
fold 2
[[42971  4159]
 [ 5729  3340]]
Accuracy : 0.8241
AUC Score (Train): 0.735492
logloss SCore: 0.848767
escape time: 3.665201425552368
fold 3
[[42686  4415]
 [ 5640  3458]]
Accuracy : 0.8211
AUC Score (Train): 0.728812
logloss SCore: 0.854995
escape time: 3.6482439041137695
fold 4
[[42562  4456]
 [ 5678  3502]]
Accuracy : 0.8197
AUC Score (Train): 0.730155
logloss SCore: 0.857884
escape time: 3.57444167137146
fold 5
[[42744  4203]
 [ 5761  3491]]
Accuracy : 0.8227
AUC Score (Train): 0.736663
logloss SCore: 0.857124
escape time: 3.692126750946045
fold 6
[[42804  4309]
 [ 5713  3373]]
Accuracy : 0.8217
AUC Score (Train): 0.728857
logloss SCore: 0.854840
escape time: 3.575438976287842
fold 7
[[42755  4215]
 [ 5

# 保存预测结果

In [95]:
save_result2csv(predictions,submit,'./round1_diac2019_test.csv')

# 数据后处理

In [ ]:
#把一年内购买次数超过10次的人，购买概率增加0.1
res = pd.read_csv('./round1_diac2019_test.csv')
id0=submit["customer_id"][submit["count"]>10]
id1=np.array(id0)
for i in trange(len(id1)):
    res["result"][res["customer_id"]==id1[i]]+=0.1

In [95]:
res.to_csv('./res_final.csv',index=False)